# Checking CREODIAS

Do they have everything we need for the Planetary Computer?
First, what do we have in the Planetary Computer?

In [5]:
from IPython.display import display
from pystac_client import Client
from rich.table import Table

client = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
collections = [
    c
    for c in client.get_collections()
    if c.id.startswith("sentinel-3") or c.id.startswith("sentinel-5p")
]
table = Table("id", "description")
for collection in collections:
    table.add_row(collection.id, collection.description[0:100] + "...")
display(table)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ id                               ┃ description                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sentinel-5p-l2-netcdf            │ The Copernicus [Sentinel-5                                                   │
│                                  │ Precursor](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel... │
│ sentinel-3-olci-wfr-l2-netcdf    │ This Collection provides Sentinel-3 Full Resolution [OLCI Level-2 Water]     │
│                                  │ products containin...                                                        │
│ sentinel-3-synergy-aod-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 Aerosol Optical     │
│                                  │ Depth](https://sentinels.co...                                               │
│ sentinel-3-synergy-v10-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 10-Day Surface      │
│                                  │ Reflectance and NDVI](https:...                                              │
│ sentinel-3-olci-lfr-l2-netcdf    │ This collection provides Sentinel-3 Full Resolution [OLCI Level-2 Land]      │
│                                  │ products containing...                                                       │
│ sentinel-3-sral-lan-l2-netcdf    │ This Collection provides Sentinel-3 [SRAL Level-2 Land                       │
│                                  │ Altimetry](https://sentinel.esa.int/web/senti...                             │
│ sentinel-3-slstr-lst-l2-netcdf   │ This Collection provides Sentinel-3 [SLSTR Level-2 Land Surface              │
│                                  │ Temperature](https://sentinel.esa.in...                                      │
│ sentinel-3-slstr-wst-l2-netcdf   │ This Collection provides Sentinel-3 [SLSTR Level-2 Water Surface             │
│                                  │ Temperature](https://sentinel.esa.i...                                       │
│ sentinel-3-sral-wat-l2-netcdf    │ This Collection provides Sentinel-3 [SRAL Level-2 Ocean                      │
│                                  │ Altimetry](https://sentinel.esa.int/web/sent...                              │
│ sentinel-3-slstr-frp-l2-netcdf   │ This Collection provides Sentinel-3 [SLSTR Level-2 Fire Radiative            │
│                                  │ Power](https://sentinel.esa.int/we...                                        │
│ sentinel-3-synergy-syn-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 Land Surface        │
│                                  │ Reflectance and Aerosol](https...                                            │
│ sentinel-3-synergy-vgp-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 Top of Atmosphere   │
│                                  │ Reflectance](https://sent...                                                 │
│ sentinel-3-synergy-vg1-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 1-Day Surface       │
│                                  │ Reflectance and NDVI](https:/...                                             │
└──────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────┘

## Sentinel 3

Let's start with Sentinel 3.
We want to make sure that every sensor and product type is in the CREODIAS API.

In [20]:
import copy
from requests import Session

SEARCH_URL = "https://datahub.creodias.eu/odata/v1/Products"
session = Session()

query = {
    "$top": 1,
    "$orderby": "PublicationDate asc",
}
for collection in collections:
    if not collection.id.startswith("sentinel-3"):
        continue
    parts = collection.id.split("-")
    collection_query = copy.deepcopy(query)
    product_type = collection.summaries.get_list("s3:product_type")[0]
    collection_query["$filter"] = (
        "Collection/Name eq 'SENTINEL-3' and "
        "PublicationDate gt 2023-05-01T00:00:00Z and "
        f"(startswith(Name, 'S3A_{product_type}') or startswith(Name, 'S3B_{product_type}'))"
    )
    response = session.get(SEARCH_URL, params=collection_query)
    print(collection.id, ":", response.json())

sentinel-3-olci-wfr-l2-netcdf : {'@odata.context': '$metadata#Products', 'value': [{'@odata.mediaContentType': 'application/octet-stream', 'Id': '322c8c70-0dd5-441b-ab8b-0a80abe305bb', 'Name': 'S3A_OL_2_WFR____20230422T020402_20230422T020702_20230422T041202_0179_098_060_2880_MAR_O_NR_003.SEN3', 'ContentType': 'application/octet-stream', 'ContentLength': 441435136, 'OriginDate': '2023-04-22T03:13:48.588Z', 'PublicationDate': '2023-05-01T00:02:56.007Z', 'ModificationDate': '2023-05-01T00:02:56.007Z', 'Online': True, 'EvictionDate': '', 'S3Path': '/eodata/Sentinel-3/OLCI/OL_2_WFR___/2023/04/22/S3A_OL_2_WFR____20230422T020402_20230422T020702_20230422T041202_0179_098_060_2880_MAR_O_NR_003.SEN3', 'Checksum': [{}], 'ContentDate': {'Start': '2023-04-22T02:04:02.279Z', 'End': '2023-04-22T02:07:02.279Z'}, 'Footprint': "geography'SRID=4326;POLYGON ((109.443 -0.120098, 110.049 -0.258904, 110.656 -0.39691, 111.26 -0.534223, 111.872 -0.673305, 112.474 -0.809957, 113.078 -0.946985, 113.682 -1.08383, 

## Sentinel 5P

Now let's do a similar exercise for the `sentinel-5p` collection.

In [28]:
collection = next(c for c in collections if c.id.startswith("sentinel-5p"))
for product_type in collection.summaries.get_list("s5p:product_type"):
    collection_query = copy.deepcopy(query)
    collection_query["$filter"] = (
        "Collection/Name eq 'SENTINEL-5P' and "
        "PublicationDate gt 2023-05-01T00:00:00Z and "
        f"startswith(Name, 'S5P_NRTI_{product_type}')"
    )
    response = session.get(SEARCH_URL, params=collection_query)
    print(collection.id, ":", product_type, ":", response.json())

sentinel-5p-l2-netcdf : L2__AER_AI : {'@odata.context': '$metadata#Products', 'value': [{'@odata.mediaContentType': 'application/octet-stream', 'Id': '139deaa3-8334-4634-b000-9741231ebfa6', 'Name': 'S5P_NRTI_L2__AER_AI_20230420T233422_20230420T233922_28596_03_020500_20230421T003803.nc', 'ContentType': 'application/octet-stream', 'ContentLength': 11131632, 'OriginDate': '2023-04-20T22:54:36.467Z', 'PublicationDate': '2023-05-01T00:09:01.295Z', 'ModificationDate': '2023-05-01T00:09:01.295Z', 'Online': True, 'EvictionDate': '', 'S3Path': '/eodata/Sentinel-5P/TROPOMI/L2__AER_AI/2023/04/20/S5P_NRTI_L2__AER_AI_20230420T233422_20230420T233922_28596_03_020500_20230421T003803', 'Checksum': [{}], 'ContentDate': {'Start': '2023-04-20T23:34:16.000Z', 'End': '2023-04-20T23:39:29.000Z'}, 'Footprint': "geography'SRID=4326;MULTIPOLYGON (((-180 -81.73773365491522, -179.94785 -81.72596, -176.0248 -80.44978, -173.08981 -79.13527, -170.8421 -77.79449, -169.08722 -76.43516, -167.69327 -75.06219, -166.57262

### Missing data

Looks like we're missing:

- `L2__CH4___`
- `L2__NP_BD3`
- `L2__NP_BD6`
- `L2__NP_BD7`
- `L2__O3_TCL`

Let's see if they're `OFFL` instead of `NRTI`.

In [29]:
missing_product_types = [
    "L2__CH4___",
    "L2__NP_BD3",
    "L2__NP_BD6",
    "L2__NP_BD7",
    "L2__O3_TCL",
]
collection = next(c for c in collections if c.id.startswith("sentinel-5p"))
for product_type in missing_product_types:
    collection_query = copy.deepcopy(query)
    collection_query["$filter"] = (
        "Collection/Name eq 'SENTINEL-5P' and "
        "PublicationDate gt 2023-05-01T00:00:00Z and "
        f"startswith(Name, 'S5P_OFFL_{product_type}')"
    )
    response = session.get(SEARCH_URL, params=collection_query)
    print(collection.id, ":", product_type, ":", response.json())

sentinel-5p-l2-netcdf : L2__CH4___ : {'@odata.context': '$metadata#Products', 'value': [{'@odata.mediaContentType': 'application/octet-stream', 'Id': '3b3ec0d8-6407-44dc-b6bd-9a198a5de6cc', 'Name': 'S5P_OFFL_L2__CH4____20230403T095605_20230403T113735_28347_03_020500_20230405T021404.nc', 'ContentType': 'application/octet-stream', 'ContentLength': 73800227, 'OriginDate': '2023-04-05T11:11:55.187Z', 'PublicationDate': '2023-05-01T01:44:34.662Z', 'ModificationDate': '2023-05-15T18:49:21.827Z', 'Online': True, 'EvictionDate': '', 'S3Path': '/eodata/Sentinel-5P/TROPOMI/L2__CH4___/2023/04/03/S5P_OFFL_L2__CH4____20230403T095605_20230403T113735_28347_03_020500_20230405T021404', 'Checksum': [{}], 'ContentDate': {'Start': '2023-04-03T10:17:39.000Z', 'End': '2023-04-03T11:16:03.000Z'}, 'Footprint': "geography'SRID=4326;POLYGON ((-101.54588 59.972004, -99.33274 61.01724, -96.96691 62.021164, -94.438705 62.979435, -91.739296 63.887047, -88.861046 64.73833, -85.79848 65.52774, -82.54951 66.24884, -79

## Conclusion

We're good!
We've got them all, or at least SOME data exists for all PC products since the start of May 2023.
We'll want to look at the `OFFL` vs `NRTI` vs `RPRO` for Sentinel-5P, but we _should_ be able to find what we need.